# Busca de Índice e Consulta de documento

**Autor:** Davi J. Leite Santos  
**Versão:** 0.0.3  
**Data:** 25 de Abril de 2024  
**Localização:** Ribeirão das Neves, Minas Gerais - Brasil  

## Contato
- 🏠 **Endereço:** Ribeirão das Neves, Minas Gerais - Brasil
- 📧 **Email:** davi.jls@outlook.com
- 🌐 **LinkedIn:** davi-j-leite-santos
- 🌐 **Website:** davijls.com.br

## Principais Competências
- **Cibersegurança**
- **Segurança da Informação**
- **Operações de TI**

# Sobre o codigo

Este código Python aborda a indexação e pesquisa de documentos com foco na eficiência e relevância. Ele utiliza ferramentas de processamento de linguagem natural (NLP) para preparar os dados e realiza pesquisas rápidas e precisas no índice, identificando e exibindo os domínios mais relevantes com pontuações calculadas com base na frequência e tempo de busca.

### Componentes Principais do Código

#### 1. **Configurações Iniciais**
- O código começa com a importação de módulos essenciais, como `json`, `re`, `nltk`, `os`, `time` e funcionalidades de `collections.Counter` e `urllib.parse.urlparse`.
- Downloads de componentes do NLTK necessários para tokenização, remoção de stopwords e stemming em português.

#### 2. **Funções de Limpeza e Processamento de Texto**
- **clean_text**: Limpa o texto removendo caracteres especiais e stopwords, convertendo tudo para minúsculas.
- **apply_stemming**: Aplica o stemming às palavras tokenizadas para reduzir a sua forma base, ajudando na indexação e na busca.
- **preprocess_query**: Encapsula o processo de limpeza e stemming em uma função destinada a preparar consultas de pesquisa.

#### 3. **Funções de Carregamento e Salvamento de Dados**
- **load_json** e **save_json**: Carregam e salvam dados em formato JSON, respectivamente, com a manipulação correta da codificação para preservar caracteres não ASCII.
- **load_data**: Carrega dados de vocabulário e índice geral previamente processados.

#### 4. **Função de Busca**
- **search_query**: Processa termos de consulta, recuperando e listando documentos relevantes com base nos termos pesquisados. Registra o tempo de busca, possibilitando avaliações de performance.

#### 5. **Análise e Apresentação de Resultados**
- **extract_domain**: Extrai o domínio de nomes de arquivo formatados, assumindo que os nomes contêm informações de domínio.
- **calculate_score**: Calcula uma pontuação para cada domínio com base na frequência de ocorrência e no tempo de busca, ponderando esses fatores para enfatizar mais a frequência.
- **display_results_with_scores**: Organiza e exibe os resultados da busca, mostrando os domínios mais relevantes juntamente com suas frequências e pontuações calculadas.

### Uso e Benefícios

- Este script é altamente eficiente para sistemas de recuperação de informação onde a precisão e a velocidade são cruciais.
- As funções de limpeza e stemming são particularmente úteis para tratar dados em português, tornando-o relevante para aplicações em idiomas além do inglês.
- A funcionalidade de pontuação e classificação de domínios é inovadora, pois combina métricas de frequência e performance de busca para destacar os resultados mais relevantes.

Este sistema é uma solução completa para busca e análise de documentos indexados, ideal para ser aplicado em contextos onde a velocidade e a relevância da informação são prioritárias, como em serviços de busca na web ou sistemas internos de gestão de conhecimento.

# ----------------------------------------------------------------

# Imoportações

In [2]:
import json
import os
import re
import nltk
import time
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from collections import Counter
from urllib.parse import urlparse

nltk.download("rslp")
nltk.download("punkt")
nltk.download("stopwords")

# Inicializar o Stemmer para português
stemmer = RSLPStemmer()
stop_words = set(stopwords.words("portuguese"))


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\davim\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\davim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\davim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Funções de Limpeza e Processamento de Texto

In [3]:
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)


def save_json(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


def clean_text(text):
    text = re.sub(r"[^\w\s]", " ", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)


def apply_stemming(text):
    tokens = nltk.word_tokenize(text, language="portuguese")
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)


def preprocess_query(query):
    cleaned_query = clean_text(query)
    stemmed_query = apply_stemming(cleaned_query)
    return stemmed_query


# Funções para Carregar Dados

In [4]:
def load_data(vocab_geral_path, index_geral_path):
    vocab_geral = load_json(vocab_geral_path)
    index_geral = load_json(index_geral_path)
    return vocab_geral, index_geral


In [5]:
# Defina os caminhos dos arquivos
vocab_geral_file = "cleaned_vocab_geral.json"
index_geral_file = "cleaned_index_geral.json"


In [6]:
# Carregar os dados apenas uma vez
vocab_geral, index_geral = load_data(vocab_geral_file, index_geral_file)


# Função de Busca

In [7]:
def search_query(query, vocab_geral, index_geral):
    # Registrar o tempo de início da busca
    start_time = time.time()

    # Processar a query
    query_terms = preprocess_query(query).split()
    query_terms = list(set(query_terms))

    # Obter IDs das palavras da query
    word_ids = []
    for term in query_terms:
        if term in vocab_geral:
            word_ids.append(vocab_geral[term])

    # Recuperar documentos correspondentes
    result_files = set()
    for word_id in word_ids:
        word_id = str(word_id)
        if word_id in index_geral:
            result_files.update(index_geral[word_id]["file_names"])

    # Registrar o tempo de término da busca
    end_time = time.time()
    search_time = end_time - start_time

    return list(result_files), search_time


In [8]:
# Exemplo de uso
query = "buceta cu vagina anal sexo toy pussy pinto dick desgraça tnc fuder sefuder fdp"
result_files, search_time = search_query(query, vocab_geral, index_geral)


# Extraindo os melhores sites

In [9]:
def extract_domain(file_name):
    parts = file_name.split("]")
    if len(parts) > 0:
        domain = parts[0].replace("_", ".")
        return domain
    return None


def calculate_score(frequency, search_time):
    # Defina os pesos para frequência e tempo de busca
    frequency_weight = 0.7
    time_weight = 0.3

    # Calcule o score ponderado
    score = (frequency * frequency_weight) - (search_time * time_weight)
    return score


def display_results_with_scores(result_files, search_time):
    domain_counter = Counter()
    for file in result_files:
        domain = extract_domain(file)
        if domain:
            domain_counter[domain] += 1

    sorted_domains = domain_counter.most_common()

    print(f"{'Site':<47}{'Frequency':<13}{'Score(ms acomulado)':<10}")
    print("=" * 80)
    for domain, frequency in sorted_domains:
        score = calculate_score(frequency, search_time)
        print(f"{domain:<47}{frequency:<13}{score:<10.2f}")


In [10]:
# Exibir resultados organizados por site com score e tempo de busca
print(f"Query que foi feita: {query}\n")
display_results_with_scores(result_files, search_time)

print(f"\nSearch completed in {search_time:.2f} seconds")


Query que foi feita: buceta cu vagina anal anal sexo toy pussy 

Site                                           Frequency    Score(ms acomulado)
www.terra.com.br                               304          212.77    
imirante.com                                   278          194.57    
emerj.tjrj.jus.br                              241          168.67    
www.metropoles.com                             179          125.27    
www.poder360.com.br                            154          107.77    
oglobo.globo.com                               142          99.37     
blogdaboitempo.com.br                          111          77.67     
mundoeducacao.uol.com.br                       111          77.67     
jovempan.com.br                                76           53.17     
glamour.globo.com                              49           34.27     
gq.globo.com                                   47           32.87     
nao-palavra.blogspot.com                       46           32.17     
sta